In [0]:
import numpy as np
import pandas as pd # data import and maniplation 
import os 
import seaborn as sns # data visualization 
import matplotlib.pyplot as plt # data visualization 
import re
from datetime import datetime, timedelta

from sklearn import datasets, linear_model # regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # data split 
from random import randint

sns.set_style("whitegrid")

In [3]:
# if I want to change between google collab or spyder 
if True:
    from google.colab import files
    files.upload()    

Saving Nepal Cumulative Burns 2014-19 copy.xlsx to Nepal Cumulative Burns 2014-19 copy.xlsx


In [0]:
df = pd.read_excel('Nepal Cumulative Burns 2014-19 copy.xlsx')

In [0]:
df.describe()

,S NO,Year,Female,Flame,Acid,Alkali,Contact,incident outside house,At work,Mortality,Deep TBSA%,%Superficial,Abdomen,RLE,Perineum,Circumferential,Silver Nitrate,Silver Applied,Acticoat Applied,Collagen Applied,Tangential Excision,Mesh 1:3,Mesh 1:4,%graft take on the day of discharge,Inadequate Excision,% Grafted.1,Mesh 1:2.1,Mesh 1:3.1,Mesh 1:4.1,Autograft,% graft take on first dressing,%graft take on 7th POD,Amputation,% graft take on 7th POD,Amputation.1,4th Excision,%Excised,% Grafted.3,Cadaveric,Live relative,%graft take on first dressing.1,% graft take on 7th POD.1,Amputation.2,%Excised .1,% Grafted.4,Autograft.3,Cadaveric .1,Live relative.1,%graft take on first dressing.2,% graft take on 7th POD.2,Type of Amputation.2,Other Comments,LAMA,Septic Shock,MDR,MDR.1,MRSA.2,MDR.2,MRSA.3,MDR.3,MRSA.4,MDR.4,DOR
count,2101.000000,2101.000000,1175.000000,1281.000000,49.000000,27.000000,52.000000,524.000000,194.000000,508.000000,1675.000000,578.000000,544.000000,1069.000000,151.000000,104.000000,6.0,43.0,2.0,148.0,1080.000000,684.000000,13.000000,16.000000,4.000,173.000000,67.000000,68.000000,19.000000,142.000000,84.000000,68.000000,21.0,8.000000,2.0,9.0,2.000000,4.000000,1.0,0.0,3.000000,2.000000,1.0,4.00000,4.00000,4.0,1.0,0.0,1.0,1.0,0.0,2.000000,3.000000,7.0,31.0,62.0,6.000000,26.0,3.0,3.0,1.0,3.0,79.0
mean,218.297001,2017.114707,0.982979,1.003123,0.510204,0.111111,0.538462,0.893130,0.907216,0.879921,19.429567,5.398054,1.159926,1.001871,0.907285,0.913462,1.0,1.0,1.0,1.0,1.000926,0.998538,0.923077,30.035625,46.500,10.665723,1.014925,1.000000,0.947368,1.007042,80.285714,78.235294,1.0,77.500000,1.0,1.0,4.500000,2.750000,1.0,NaN,81.666667,87.500000,1.0,4.75000,4.75000,1.0,1.0,NaN,80.0,80.0,NaN,8.500000,3.666667,1.0,1.0,1.0,1.333333,1.0,1.0,1.0,1.0,1.0,1.0
std,152.364554,1.362658,0.129406,0.230467,0.505076,0.320256,0.503382,0.309243,0.290879,0.325374,18.901897,8.196488,2.286596,0.317994,0.290998,0.282519,0.0,0.0,0.0,0.0,0.030429,0.038236,0.277350,44.628814,53.157,13.004125,0.122169,0.172774,0.229416,0.083918,20.144999,24.592268,0.0,18.898224,0.0,0.0,0.707107,2.061553,NaN,NaN,12.583057,10.606602,NaN,3.40343,3.40343,0.0,NaN,NaN,NaN,NaN,NaN,6.363961,4.618802,0.0,0.0,0.0,0.816497,0.0,0.0,0.0,NaN,0.0,0.0
min,1.000000,2014.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.000000,0.000000,0.000000,0.850000,0.000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.0,50.000000,1.0,1.0,4.000000,1.000000,1.0,NaN,70.000000,80.000000,1.0,2.00000,2.00000,1.0,1.0,NaN,80.0,80.0,NaN,4.000000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
25%,88.000000,2016.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,5.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.972500,0.750,2.000000,1.000000,1.000000,1.000000,1.000000,80.000000,80.000000,1.0,65.000000,1.0,1.0,4.250000,1.000000,1.0,NaN,75.000000,83.750000,1.0,2.00000,2.00000,1.0,1.0,NaN,80.0,80.0,NaN,6.250000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
50%,193.000000,2017.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,14.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,45.500,7.000000,1.000000,1.000000,1.000000,1.000000,90.000000,85.000000,1.0,85.000000,1.0,1.0,4.500000,2.500000,1.0,NaN,80.000000,87.500000,1.0,4.00000,4.00000,1.0,1.0,NaN,80.0,80.0,NaN,8.500000,1.000000,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
75%,323.000000,2018.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,30.000000,7.750000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,90.000000,91.250,15.000000,1.000000,1.000000,1.000000,1.000000,92.750000,95.000000,1.0,91.250000,1.0,1.0,4.750000,4.250000,1.0,NaN,87.500000,91.250000,1.0,6.75000,6.75000,1.0,1.0,NaN,80.0,80.0,NaN,10.750000,5.000000

In [28]:
df.head()

,S NO,Year,Program,Age (yrs),Male,Female,Address,Baux Score,ABSI,Flame,Lightning,Scald,Electrical,Acid,Alkali,Steam,Contact,Others,incident inside house,At cooking place,incident outside house,At work,Suicide,Homicide,Date /Time of burn injury,Time of Burn,Date/Time of admission,Time of Admission,Time Elapsed,Date of Discharge,ALL Total Hosp Stay,Mortality,cause of death/day of death(post burn),Date of Death,Post burn Day of Death,weight at admission,weight at discharge,Weight Loss,Total days in ICU,Mechanical Ventilation,...,2nd Culture,Date.5,Organism.3,Sensitive.2,Resistance.2,MRSA.2,MDR.2,3RD C/S,date.1,Organism.4,Sensitivec,Resistance.3,MRSA.3,MDR.3,4RD C/S,Date.6,Organism.5,Sensitive.3,Resistance.4,MRSA.4,MDR.4,5RD C/S,Date.7,Organism.6,Sensitive.4,Resistance.5,6RD C/S,Date.8,Organism.7,Sensitive.5,Resistance.6,7RD C/S,Date.9,Organism.8,Sensitive.6,Resistance.7,LAMA.1,DOR,Septic shock,patient
0,1,2014,resurge,6,1,NaN,ramroshan accham,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,12th dec 2013,NaN,26th dec.2013,NaN,14,11th jan. 2014,14,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,2014,resurge,23,1,NaN,"thecho, lalitpur",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,8th dec 2013(23 mangsir),NaN,5th jan.2014,NaN,17,12th jan.2014,7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,3,2014,resurge,33,NaN,1.0,"nayabazar, balaju",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,4th jan 2014,NaN,"4th jan 2014,12:25 pm",NaN,0,21st jan 2014,17,0.0,NaN,NaN,NaN,45,43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,4,2014,resurge,58,1,NaN,"gothatar-2,ichhar billigal, kathmandu",NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,"4th dec(19 mangsir,2070)",NaN,12th jan 2014,NaN,38,14th jan.2014,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,5,2014,os,42,NaN,1.0,lalitpur,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,28th dec,NaN,"9th jan,2014, 4 pm",NaN,11,29th jan2014,19,0.0,NaN,NaN,NaN,76,75,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [0]:
# gives a patient number proxy 
df['patient'] = np.arange(len(df))

Things to do
- consolidate gender 
- standarize categorical numbers 
- Address
- dates 

In [15]:
df['Program'].unique()

array(['ReSurge', 'OS', 'Self', 'WW', 'SELF', nan, 'Resurge', 'self',
       'RISOP/CMAF', 'RISOP', 'CMAF', 'IOM', 'EQ', 'Risop', 'Self/RISOP',
       'Wonderwork', 'RiSOP', 'UNICEF', 'Pay later', 'R', 'Silf',
       'self/RISOP', 'Care of Lahan', 'Wonder Work', 'Sathi Sewa',
       'Italian team', 'Self ', 'self ', 'sef', 'selff', 'Self1'],
      dtype=object)

In [0]:
df['Program'] = df['Program'].apply(lambda x: str(x).lower().strip().replace('silf', 'self').replace('wonder work', 'ww').replace('wonderwork', 'ww').replace('sef', 'self').replace('selff', 'self').replace('self1', 'self'))

In [0]:
df['Address'] = df['Address'].apply(lambda x: str(x).lower().strip().replace('accham', 'achham').replace('agrakhanchi', 'arghakhanchi').replace('argakhachi', 'arghakhanchi').replace('argakhanchi', 'arghakhanchi').replace('arghakhachi', 'arghakhanchi').replace('baglong', 'baglung').replace('bajhang', 'bajang').replace('bhairawa', 'bhairahawa').replace('bharatpur','bhaktapur').replace('bhkatapur','bhaktapur').replace('butuwal','butwal').replace('bajang','bajhang').replace('chayanpur','chainpur').replace('dailekha','dailekh').replace('dhangadi','dhangari').replace('dhanusa','dhanusha').replace('dhanushaa','dhanusha').replace('dhulekhel','dhulikhel').replace('dolakaha','dolakha').replace('dolkaha','dolakha').replace('illam','ilam').replace('ithari','itahari').replace('ittahari','itahari').replace('kapilbastu','kapilvastu').replace('kavrepalanchowk','kavrepalanchok').replace('kavre','kavrepalanchok').replace('kirtipipur','kirtipur').replace('kohalpir','kohalpur').replace('kuleshor','kuleshwor').replace('lamgunj','lamjung').replace('llitpur','lalitpur').replace('mahatori','mahottari').replace('mahatari','mahottari').replace('mahotari','mahottari').replace('makawanpur','makwanpur').replace('nawalparashi','nawalparasi').replace('nawalparasi','nawalparasi').replace('nepaljung','nepalgunj').replace('okahaldhunga','okhaldhunga').replace('okaldhunga','okhaldhunga').replace('parsha','parsa').replace('pokahara','pokhara').replace('ramechap','ramechhap').replace('samkhuwasabha','sankhuwasabha').replace('sankhuwasava','sankhuwasabha').replace('sakhawasaba','sankhuwasabha').replace('sangjhya','sanghiya').replace('saralahi','sarlahi').replace('shankhuwashabha','sankhuwasabha').replace('shyanja','shyangja').replace('simra','simara').replace('simra','simara').replace('sindhui','sindhuli').replace('sindhupalchowk','sindhupalchok').replace('sinduli','sindhuli').replace('sindupalchok','sindhupalchok').replace('sindupalchowk','sindhupalchok').replace('snkhuwasabha','sankhuwasabha').replace('syanja','syangja').replace('syanjha','syangja').replace('tanahu','tanahun').replace('taplegunj','taplejung').replace('taudhaha','taudaha').replace('terathum','tehrathum').replace('udaypur','udayapur'))

In [33]:
df['Address'].unique()

array(['ramroshan achham', 'thecho, lalitpur', 'nayabazar, balaju',
       'gothatar-2,ichhar billigal, kathmandu', 'lalitpur', 'malekhu',
       'udayapur', 'surkhet', 'mahadevpuri-7 banke', '12-chitwan',
       'khotang', 'dang', 'birgunj', 'hetauda', 'ranibari sarlahi',
       'gurujudhara 2 kathmandu', 'dumarwana 4 sundarbasti, bara',
       'nepalgunj', 'rautahat', 'sarlahi', 'kanchanpur', 'sindhuli',
       'dadeldhura', 'bara', 'dhanusha', 'kathmandu', 'lamjung',
       'bhaktapur', 'sindhupalchok', 'pyuthan', 'morang', 'jumla',
       'bajhang', 'santapur 3 rautahat', 'gulmi', '12- kritipur',
       'bardiya', 'nuwakot', 'gothi,humla', 'kirtipur', 'siraha', 'parsa',
       'new road/ mumbai', 'dhading', 'chitwan', 'bhairahawa',
       'panchthar', 'taplejung', 'salyan', 'rukum', 'dolkha',
       'mahendranagar', 'dhangari', 'janakpur', 'ktm', 'okhaldhunga',
       'ilam', 'tanahun', 'itahari', 'nawalparasi', 'dailekh', 'damauli',
       'butwal', 'kavrepalanchok', 'dolakha', 'k